### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [149]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load  = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [150]:
# Make sure every cell of the data is filled
purchase_data.count()
# Every column has 780 rows so there is no empty cells in the data

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [151]:
# Notes: 1 Player can buy multiple games
players_df    = purchase_data['SN']
total_players = len(players_df.value_counts())

print("The total number of players is: " + str(total_players))
players_df.value_counts()

The total number of players is: 576


Lisosia93       5
Iral74          4
Idastidru52     4
Saedaiphos46    3
Siallylis44     3
               ..
Marughi89       1
Hailaphos89     1
Yasur85         1
Iduelis31       1
Lirtassa52      1
Name: SN, Length: 576, dtype: int64

## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [152]:
# * Number of Unique Items
unique_values = purchase_data['Item Name'].value_counts()
number_of_unique_items = len(unique_values)
# * Average Purchase Price
average_purchase_price = purchase_data['Price'].mean()
# * Total Number of Purchases
total_number_of_purchases = len(purchase_data['Purchase ID'])
# * Total Revenue
total_revenue = purchase_data['Price'].sum()

print("The     Total     Revenue      is: "    + str("%.2f" % total_revenue))
print("The  Total Number of Purchases is:  "   + str("%.2f" % total_number_of_purchases))
print("The  Number  of  Unique Items  is:  "   + str("%.2f" % number_of_unique_items)) 
print("The  Average  Purchase  Price  is:    " + str("%.2f" % average_purchase_price))


The     Total     Revenue      is: 2379.77
The  Total Number of Purchases is:  780.00
The  Number  of  Unique Items  is:  179.00
The  Average  Purchase  Price  is:    3.05


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [244]:
# Create a new dataframe with: ID  Gender  totalSpent
aggregation_functions = {'Price': 'sum', 'Gender': 'first'}
filtered_df = purchase_data.groupby(purchase_data['SN']).aggregate(aggregation_functions)
genders_series = filtered_df['Gender'].value_counts()

# Divide the number of each gender by the total player for percentages
gender_percentages = round((genders_series / total_players) * 100,3)

print("The  male  % is: " + str(gender_percentages['Male'])   + '%')
print("The female % is: " + str(gender_percentages['Female']) + '%')
print("The Other / Non-Disclosed % is: " + str(gender_percentages['Other / Non-Disclosed']) + '%')
genders_series

The  male  % is: 84.028%
The female % is: 14.062%
The Other / Non-Disclosed % is: 1.91%


Male                     484
Female                    81
Other / Non-Disclosed     11
Name: Gender, dtype: int64


## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [250]:
#   * The below each broken by gender
filtered_df["Total Games"] = players_df.value_counts()
#   * Purchase Count
aggregation_functions = {'Price': 'sum', 'Total Games': 'sum'}
purchase_count_by_gender = filtered_df.groupby(filtered_df['Gender']).aggregate(aggregation_functions)
#   * Average Purchase Total per Person by Gender
purchase_count_by_gender['Average Purchase'] = pd.Series({'Female':" %.2f" % (purchase_count_by_gender.at['Female', 'Price'] / genders_series['Female']),
                                                          'Male'  :" %.2f" % (purchase_count_by_gender.at['Male', 'Price']   / genders_series[ 'Male' ]),
                                                          'Other / Non-Disclosed':" %.2f" % (purchase_count_by_gender.at['Other / Non-Disclosed', 'Price'] / genders_series['Other / Non-Disclosed']),
                                                         })
#   * Average Purchase Price
filtered_df['Price'].mean()
#   * Total Purchase Value
total_purchase_value = purchase_count['Price'].sum()

print("Purchase count by gender:\n",purchase_count_by_gender , "\n")
print("Average Purchase Price:  " + str(" %.2f" % filtered_df['Price'].mean()))
print("Total Purchase Value: "    + str(" %.2f" % total_purchase_value))




Purchase count by gender:
                          Price  Total Games Average Purchase
Gender                                                      
Female                  361.94          113             4.47
Male                   1967.64          652             4.07
Other / Non-Disclosed    50.19           15             4.56 

Average Purchase Price:   4.13
Total Purchase Value:  2379.77


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [61]:
### Age Demographics

#   * The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
#   * Purchase Count
#   * Average Purchase Price
#   * Total Purchase Value
#   * Average Purchase Total per Person by Age Group

20    99
22    70
24    67
23    67
21    62
25    59
15    35
30    35
16    30
18    26
19    23
17    22
35    14
33    14
26    14
29    13
27    10
10     9
7      9
34     9
38     9
32     8
8      8
11     7
31     7
37     7
39     6
40     6
9      6
12     6
36     5
28     5
13     4
44     2
41     2
14     2
43     1
42     1
45     1
Name: Age, dtype: int64

## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [68]:
purchase_data.head(10)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
5,5,Yalae81,22,Male,81,Dreamkiss,3.61
6,6,Itheria73,36,Male,169,"Interrogator, Blood Blade of the Queen",2.18
7,7,Iskjaskst81,20,Male,162,Abyssal Shard,2.67
8,8,Undjask33,22,Male,21,Souleater,1.10
9,9,Chanosian48,35,Other / Non-Disclosed,136,Ghastly Adamantite Protector,3.58


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame

